In [ ]:
!pip -q install -U "transformers>=4.40.0" "datasets" "peft" "bitsandbytes" "accelerate" "scikit-learn" "evaluate"
import os, sys, json, torch, pandas as pd
print("Torch :", torch.__version__, "| GPU:", torch.cuda.get_device_name(0))

Torch : 2.6.0+cu124 | GPU: Tesla T4


In [ ]:
!pip install -q --upgrade transformers

In [ ]:
from google.colab import files
uploaded = files.upload()
csv_path = next(iter(uploaded))

Saving labeled_data.csv to labeled_data.csv


In [ ]:
import pandas as pd
import re
import html

# Load data
df = pd.read_csv(csv_path, usecols=['tweet', 'class']).dropna()
df.drop_duplicates(subset='tweet', inplace=True)

def clean_text(text):
    text = html.unescape(text)                                        # decode HTML entities (e.g. &#128514;)
    text = text.lower()                                               # lowercase
    text = re.sub(r'https?://\S+', '', text)                          # remove URLs
    text = re.sub(r'@\w+', '', text)                                  # remove mentions
    text = re.sub(r'#\w+', '', text)                                  # remove hashtags
    text = re.sub(r'\brt\b', '', text)                                # remove retweet tags
    text = re.sub(r'[^a-z0-9\s]', '', text)                           # remove punctuation/symbols
    text = re.sub(r'\s+', ' ', text).strip()                          # collapse spaces
    return text

# 3. Apply cleaning and drop too-short tweets
df['tweet_clean'] = df['tweet'].map(clean_text)
df = df[df['tweet_clean'].str.len() > 5]

# 4. Map numeric labels to text
label_map = {0: "hate_speech", 1: "offensive", 2: "neutral"}
df['label_text'] = df['class'].map(label_map)

# 5. Stratified sample (1000 per class)
n_per_class = 1000
df_small = (
    df.groupby('class', group_keys=False)
      .apply(lambda grp: grp.sample(n=min(len(grp), n_per_class), random_state=42))
      .reset_index(drop=True)
)

# 6. Shuffle samples
df_small = df_small.sample(frac=1, random_state=42).reset_index(drop=True)

# 7. Confirm balance and preview
print(df_small['class'].value_counts())
df_small[['tweet_clean', 'class', 'label_text']].head()


class
1    1000
0    1000
2    1000
Name: count, dtype: int64


<ipython-input-5-6305ac707e9b>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_text'] = df['class'].map(label_map)
<ipython-input-5-6305ac707e9b>:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: grp.sample(n=min(len(grp), n_per_class), random_state=42))


,tweet_clean,class,label_text
0,look at what you just said lls new era girl by...,1,offensive
1,look at yall bitches only hmu when ya wifi on ...,1,offensive
2,spongebob out here beating bitches up,1,offensive
3,we hate you too lil nicca lol i hate the steel...,0,hate_speech
4,i am a zebra amongst the sheep,2,neutral


In [ ]:
from datasets import Dataset


def row2dict(row):
    return {
        "instruction": "Classify the tweet into hate_speech, offensive, or neutral.",
        "input": row["tweet_clean"],
        "output": row["label_text"],
    }

# Convert DataFrame rows into dicts
data_list = df_small.apply(row2dict, axis=1).tolist()
dataset = Dataset.from_list(data_list)

# Train/test split
dataset = dataset.train_test_split(test_size=0.2, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 600
    })
})

In [ ]:
import random

print("Random Train Samples:")
for i in random.sample(range(len(dataset["train"])), 3):
    print(dataset["train"][i], "\n")

print("Random Test Samples:")
for i in random.sample(range(len(dataset["test"])), 3):
    print(dataset["test"][i], "\n")


Random Train Samples:
{'instruction': 'Classify the tweet into hate_speech, offensive, or neutral.', 'input': 'hes just a cunt and nobody likes him he should choke on shit', 'output': 'offensive'} 

{'instruction': 'Classify the tweet into hate_speech, offensive, or neutral.', 'input': 'bitches on my dick but look at me im fucking beautiful', 'output': 'offensive'} 

{'instruction': 'Classify the tweet into hate_speech, offensive, or neutral.', 'input': 'if u dont buy the box of colored goldfish you r a racist tbh', 'output': 'neutral'} 

Random Test Samples:
{'instruction': 'Classify the tweet into hate_speech, offensive, or neutral.', 'input': 'white bus drivers are all white trash', 'output': 'hate_speech'} 

{'instruction': 'Classify the tweet into hate_speech, offensive, or neutral.', 'input': 'cant wait 2 see my bitch tmrw', 'output': 'offensive'} 

{'instruction': 'Classify the tweet into hate_speech, offensive, or neutral.', 'input': 'stfu u ugly dyke', 'output': 'hate_speech'}

In [ ]:
!pip install -q huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "Qwen/Qwen2.5-3B"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_auth_token=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_cfg,
    trust_remote_code=True,
    device_map="auto",
    use_auth_token=True,
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Sliding Win

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Tokenization
max_len = 256

def tokenize(example):
    prompt = f"{example['instruction']}\nTweet: {example['input']}\nAnswer:"
    tokens = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=max_len,
    )
    # tokenize the target label text
    label_enc = tokenizer(
        example['output'],
        truncation=True,
        padding="max_length",
        max_length=10,
    )
    tokens["labels"] = label_enc["input_ids"]
    return tokens

# Map and remove the original columns
tokenized = dataset.map(
    tokenize,
    batched=False,
    remove_columns=["instruction", "input", "output"],
)

print(tokenized)


Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 600
    })
})


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.0597


In [ ]:
# Training arguments & Trainer
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits[..., -1, :], axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

args = TrainingArguments(
    output_dir="qwen-3b-hate-qlora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=25,

    eval_strategy="steps",
    eval_steps=500,         # evaluate every 500 steps

    save_strategy="steps",
    save_steps=500,         # save every 500 steps
)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    compute_metrics=compute_metrics,
)
trainer.train()


<ipython-input-16-d101661752b3>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: netflixahmed21 (netflixahmed21-mounir) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


TrainOutput(global_step=450, training_loss=2.0088013627794052, metrics={'train_runtime': 1873.4852, 'train_samples_per_second': 3.843, 'train_steps_per_second': 0.24, 'total_flos': 3.0707162284032e+16, 'train_loss': 2.0088013627794052, 'epoch': 3.0})

In [ ]:
trainer.save_model("qwen-3b-hate-qlora-adapter")
tokenizer.save_pretrained("qwen-3b-hate-qlora-adapter")

('qwen-3b-hate-qlora-adapter/tokenizer_config.json',
 'qwen-3b-hate-qlora-adapter/special_tokens_map.json',
 'qwen-3b-hate-qlora-adapter/vocab.json',
 'qwen-3b-hate-qlora-adapter/merges.txt',
 'qwen-3b-hate-qlora-adapter/added_tokens.json',
 'qwen-3b-hate-qlora-adapter/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r qwen-3b-hate-qlora/ /content/drive/MyDrive/

In [ ]:
!pip install -q gradio transformers peft bitsandbytes

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import gradio as gr
from google.colab import drive

drive.mount('/content/drive')

ADAPTER_PATH = "/content/drive/MyDrive/qwen-3b-hate-qlora-adapter"
BASE_MODEL   = "Qwen/Qwen2.5-3B"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_cfg,
    trust_remote_code=True,
    device_map="auto"
)

# Attach the QLoRA adapter
model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_PATH,
    is_trainable=False
)
model.eval()

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    ADAPTER_PATH,
    trust_remote_code=True
)

# Define the inference function
def classify_tweet(text: str) -> str:
    prompt = (
        "Classify the tweet into hate_speech, offensive, or neutral.\n"
        f"Tweet: {text}\n"
        "Answer:"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=11,
            do_sample=False,
        )
    # Remove the prompt tokens and strip whitespace/punctuation
    gen_tokens = out[0][ inputs["input_ids"].shape[-1] :]
    generated = tokenizer.decode(gen_tokens, skip_special_tokens=True)
    return generated.strip().rstrip(".")  # strip trailing period


# Gradio
iface = gr.Interface(
    fn=classify_tweet,
    inputs=gr.Textbox(lines=3, placeholder="Enter a tweet here...", label="Tweet"),
    outputs=gr.Textbox(label="Predicted Label"),
    title="Hate Speech Classifier",
    description="Type or paste a tweet, then click Submit to see whether it's hate_speech, offensive, or neutral."
)
iface.launch(share=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.l

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a18b874bd680cbdf56.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# ⬇️ cell 10 – Save & (optionally) push to HF Hub
model.save_pretrained("qwen-3b-hate-qlora-adapter")
tokenizer.save_pretrained("qwen-3b-hate-qlora-adapter")
# To upload:
# from huggingface_hub import login, HfApi
# login(token=YOUR_TOKEN)
# api = HfApi().upload_folder("YOUR_USERNAME/qwen-3b-hate-qlora", "qwen-3b-hate-qlora-adapter")
